In [ ]:
from itertools import izip
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import dill

In [ ]:
cd ~/g/projects/black_spruce/

In [ ]:
all_graphs = !ls *graph*all*.txt
needle_graphs = !ls *graph*needle*.txt
cambium_graphs = !ls *graph*cambium*.txt


In [ ]:
graphs = {"All":all_graphs,
         "Needle":needle_graphs,
         "Cambium":cambium_graphs}

In [ ]:
gentables = dill.load(open("gentables.dill"))

In [ ]:
sns.set_context("notebook", font_scale=1.2)

In [ ]:
def plot_pie(plot_data, title, merged):
    unique_seqs = set()
    sig = ""
    fig = plt.gcf()
    fig.set_size_inches(8,8)
    plot_data = pd.DataFrame(plot_data.ix[0:20,:])
    labels = None
    colors = sns.color_palette("hls", len(plot_data))
    colors = sns.color_palette("Blues_r", len(plot_data))
    colors = sns.color_palette("cubehelix", len(plot_data))
    colors = sns.cubehelix_palette(len(plot_data), reverse=True)
    colors = sns.cubehelix_palette(len(plot_data), 
                                   start=.5, 
                                   rot=-1.5, 
                                   dark=.15, 
                                   light=1.0, 
                                   reverse=True)
    if merged:
        plot_data['Label'] = plot_data.apply(lambda row: "%s (%d)" % (row["Term_x"],row["#Seqs"]), axis=1)
        plot_data['sig1'] = plot_data.apply(lambda x: "*" if float(x.classicFisher) < 0.05 else "", axis=1)
        plot_data['sig2'] = plot_data.apply(lambda x: "*" if x.bh == True else "", axis=1)
        plot_data['siglabel'] = plot_data.apply(lambda row: "%s%s%s" % (row.Label, row.sig1, row.sig2), axis=1)
        labels = plot_data['siglabel']
    else:
        plot_data['Label'] = plot_data.apply(lambda row: "%s (%d)" % (row["Term"],row["#Seqs"]), axis=1)
        labels = plot_data['Label']

    plt.pie(plot_data["#Seqs"],
           colors=colors,
           labels = labels)
    
    for row in plot_data.iterrows():
        for seq in row[1].Sequences.split(","):
            unique_seqs.add(seq)
    
    plt.title("%s (%d)%s"  % (title, len(unique_seqs), sig))

    plt.show()

In [ ]:
plotted = 0
for key, files in graphs.items():
    if key:
        print "================= Charts for %s =================" % key
        for g in files:
            ontology = g[-6:-4].upper()
            df = pd.read_csv(g, sep="\t", header=0, index_col=1)
            genkey = "%s-%s" % (key.lower(), ontology)
            merged = False
            if genkey in gentables:
                gt = gentables[genkey]['gt']
                gt.index = gt['GO.ID']
                df = df.merge(gt, left_index=True, right_index=True)
                merged = True
            for level, data in df.groupby("Level"):
                if level > 1:
                    d = data.sort("#Seqs", ascending=False)
                    plot_pie(d, "%s/%s/Level %d" % (key, ontology, level), merged)
                    plotted += 1

In [ ]:
plotted